In [70]:
from gensim.models import word2vec,Word2Vec
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize

In [71]:
test = pd.read_json("test-data.json","r", encoding="utf8")
train = pd.read_json("train-data.json","r", encoding="utf8")

In [72]:
df = pd.concat([test,train])

In [73]:
df.head()

,_id,corrected_question,intermediary_question,sparql_query,sparql_template_id
0,1701,Which architect of Marine Corps Air Station Ka...,What is the <architect> of the <Marine Corps A...,SELECT DISTINCT ?uri WHERE { <http://dbpedia....,16
1,3293,Which sects people are part of local political...,What is the <faith> of the <political parties>...,SELECT DISTINCT ?uri WHERE { ?x <http://dbpedi...,305
2,2161,List common systems are developed by of the Go...,What is the <developed by> of the <Google Vide...,SELECT DISTINCT ?uri WHERE { <http://dbpedia....,16
3,1136,For which games are Sam Loyd and Eric Schiller...,What is the <known for> of the <Sam Loyd> and ...,SELECT DISTINCT ?uri WHERE { <http://dbpedia.o...,15
4,987,Is Peter Piper Pizza in the pizza industry?,Is <Peter Piper Pizza> the <field of exercise>...,ASK WHERE { <http://dbpedia.org/resource/Peter...,152


In [74]:
df.shape

(5000, 5)

In [75]:
test.shape

(1000, 5)

In [81]:
def preprocess_label(review):
    review_text = review
    words = review_text.split()
    return (words)
def labels_to_sentences(label):
    raw_sentences = sent_tokenize(label.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(preprocess_label(raw_sentence))
    return sentences

In [85]:
sentences = []
for x in df.sparql_query.apply(str):
    sentences += labels_to_sentences(x)
len(sentences)    

10162

In [87]:
sentences

[['SELECT',
  'DISTINCT',
  '?uri',
  'WHERE',
  '{',
  '<http://dbpedia.org/resource/Marine_Corps_Air_Station_Kaneohe_Bay>',
  '<http://dbpedia.org/property/architect>',
  '?uri.'],
 ['<http://dbpedia.org/resource/New_Sanno_Hotel>',
  '<http://dbpedia.org/ontology/tenant>',
  '?uri}'],
 ['SELECT',
  'DISTINCT',
  '?uri',
  'WHERE',
  '{',
  '?x',
  '<http://dbpedia.org/property/international>',
  '<http://dbpedia.org/resource/Muslim_Brotherhood>',
  '.'],
 ['?x', '<http://dbpedia.org/ontology/religion>', '?uri', '.'],
 ['?x',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://dbpedia.org/ontology/PoliticalParty>}'],
 ['SELECT',
  'DISTINCT',
  '?uri',
  'WHERE',
  '{',
  '<http://dbpedia.org/resource/Google_Videos>',
  '<http://dbpedia.org/property/developer>',
  '?uri.'],
 ['<http://dbpedia.org/resource/Google_Web_Toolkit>',
  '<http://dbpedia.org/ontology/author>',
  '?uri}'],
 ['SELECT',
  'DISTINCT',
  '?uri',
  'WHERE',
  '{',
  '<http://dbpedia.org/resource/Sam_Lo

In [88]:
num_features = 300  
min_word_count = 1
num_workers = 4     
context = 10       
downsampling = 1e-3 # (0.001) 

from gensim.models import word2vec
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling,
                          sg = 1
                         )

model.init_sims(replace=True)

In [89]:
model.wv.vectors.shape

(4772, 300)

In [94]:
model.wv.most_similar("<http://dbpedia.org/ontology/Musical>}")

[('<http://dbpedia.org/ontology/Artist>}', 0.8786841630935669),
 ('<http://dbpedia.org/ontology/Place>}', 0.8638293743133545),
 ('<http://dbpedia.org/ontology/Book>}', 0.859697699546814),
 ('<http://dbpedia.org/ontology/Settlement>}', 0.8575937747955322),
 ('<http://dbpedia.org/ontology/Bank>}', 0.8538920879364014),
 ('<http://dbpedia.org/ontology/Horse>}', 0.8367944955825806),
 ('<http://dbpedia.org/ontology/SoccerPlayer>}', 0.8293639421463013),
 ('<http://dbpedia.org/ontology/GovernmentAgency>}', 0.8263630867004395),
 ('<http://dbpedia.org/ontology/FictionalCharacter>}', 0.8187824487686157),
 ('<http://dbpedia.org/ontology/Food>}', 0.8147705793380737)]

In [95]:
model.wv.most_similar("WHERE")

[('COUNT(?uri)', 0.9707749485969543),
 ('.', 0.9605094194412231),
 ('?x', 0.9590908885002136),
 ('?uri.', 0.9587105512619019),
 ('?uri', 0.9510945081710815),
 ('}', 0.9477623701095581),
 ('DISTINCT', 0.9457318186759949),
 ('{?uri', 0.9445152878761292),
 ('{', 0.9363819360733032),
 ('SELECT', 0.9273197054862976)]

In [96]:
model.wv.most_similar("SELECT")

[('.', 0.9743200540542603),
 ('COUNT(?uri)', 0.9712892770767212),
 ('?uri.', 0.9697246551513672),
 ('}', 0.9687084555625916),
 ('?x', 0.9641491770744324),
 ('DISTINCT', 0.9571288824081421),
 ('{', 0.9557734727859497),
 ('{?uri', 0.954651951789856),
 ('?uri', 0.9470460414886475),
 ('<http://dbpedia.org/ontology/award>', 0.9376944303512573)]